In [1]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline
import numpy as np
import sympy
import scipy.sparse.linalg
from math import pi

import setup_grids
from core.constit import fourth_order_tensor
from core.bc import bc
from fvdiscr import mpsa, fvutils
from viz import cell_data

In [4]:
# Homogeneous media

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 4
pert = 0.5
### End of parameter definitions

# Permeability tensor, scalar for simplicity
mu = 1
lmbda = 1

# Analytical solution
x, y = sympy.symbols('x y')
ux = sympy.sin(x) * sympy.cos(y)
uy = sympy.sin(x) * x**2
ux_f = sympy.lambdify((x, y), ux, 'numpy')
uy_f = sympy.lambdify((x, y), uy, 'numpy')
dux_x = sympy.diff(ux, x)
dux_y = sympy.diff(ux, y)
duy_x = sympy.diff(uy, x)
duy_y = sympy.diff(uy, y)
divu = dux_x + duy_y

sxx = 2 * mu * dux_x + lmbda * divu
sxy = mu * (dux_y + duy_x)
syx = mu * (duy_x + dux_y)
syy = 2 * mu * duy_y + lmbda * divu

sxx_f = sympy.lambdify((x, y), sxx, 'numpy')
sxy_f = sympy.lambdify((x, y), sxy, 'numpy')
syx_f = sympy.lambdify((x, y), syx, 'numpy')
syy_f = sympy.lambdify((x, y), syy, 'numpy')

rhs_x = sympy.diff(sxx, x) + sympy.diff(syx, y)
rhs_y = sympy.diff(sxy, x) + sympy.diff(syy, y)
rhs_x_f = sympy.lambdify((x, y), rhs_x, 'numpy')
rhs_y_f = sympy.lambdify((x, y), rhs_y, 'numpy')


def run_convergence(grid_type):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        mu_c = mu * np.ones(g.num_cells)
        lmbda_c = lmbda * np.ones(g.num_cells)
        k = fourth_order_tensor.FourthOrderTensor(2, mu_c, lmbda_c)

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.get_boundary_faces()
        bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
        
        # MPFA discretization, and system matrix
        stress, bound_stress = mpsa.mpsa(g, k, bound_cond)
        div = fvutils.vector_divergence(g)
        a = div * stress
        
        # Boundary conditions
        xf = g.face_centers
        u_bound = np.zeros((g.dim, g.num_faces))
        u_bound[0, bound_faces] = ux_f(xf[0, bound_faces], xf[1, bound_faces])
        u_bound[1, bound_faces] = uy_f(xf[0, bound_faces], xf[1, bound_faces])
        
        
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        rhs = np.vstack((rhs_x_f(xc[0], xc[1]), rhs_y_f(xc[0], xc[1]))) * g.cell_volumes
        b = rhs.ravel('F') + div * bound_stress * u_bound.ravel('F')

        # Solve system, derive fluxes
        u_num = scipy.sparse.linalg.spsolve(a, b)
        stress_num = stress * u_num - bound_stress * u_bound.ravel('F')

        ux_num = u_num[::2]
        uy_num = u_num[1::2]
        
        stress_x_num = stress_num[::2]
        stress_y_num = stress_num[1::2]
        # Exact solution
        ux_ex = ux_f(xc[0], xc[1])
        uy_ex = uy_f(xc[0], xc[1])
        u_ex = np.vstack((ux_ex, uy_ex))
        u_diff = np.vstack((ux_num - ux_ex, uy_num - uy_ex))
        
        sx_ex_faces = np.vstack((sxx_f(xf[0], xf[1]), sxy_f(xf[0], xf[1])))
        sy_ex_faces = np.vstack((syx_f(xf[0], xf[1]), syy_f(xf[0], xf[1])))
        
        stress_x_ex = np.sum(g.face_normals * sx_ex_faces, axis=0)
        stress_y_ex = np.sum(g.face_normals * sy_ex_faces, axis=0)
        stress_diff = np.vstack((stress_x_num - stress_x_ex, 
                                 stress_y_num - stress_y_ex))
        stress_ex = np.vstack((stress_x_ex, stress_y_ex))
        
        u_err.append(np.sqrt(np.sum(g.cell_volumes * u_diff**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * stress_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * stress_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']

for gr in grids:
    u, f = run_convergence(gr)
    print(u)
    print(f)


[0.013141424612606607, 0.0039595549753802251, 0.0010317052894505648, 0.00024384620727657417]
[0.0361872857904958, 0.015764291256799387, 0.0053879362435560055, 0.0020697006664007543]
[0.019724399988563384, 0.0061355046194719685, 0.0015755518569342578, 0.00041613048050742439]
[0.07735815149722608, 0.030932401116354872, 0.012835670048538602, 0.0051826294978424914]


In [3]:
# Heterogeneous media media

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 4
pert = 0.5

kappa_list = [1e-6, 1, 1e6]
### End of parameter definitions

# Permeability tensor, scalar for simplicity
mu = 1
mbda = 1

def chi(x, y):
    return np.logical_and(np.greater(x, 0.5), np.greater(y, 0.5))

# Analytical solution
x, y = sympy.symbols('x y')
ux = sympy.sin(2*pi * x) * sympy.sin(2 * pi * y)
uy = sympy.cos(pi * x) * (y-0.5)**2
ux_f = sympy.lambdify((x, y), ux, 'numpy')
uy_f = sympy.lambdify((x, y), uy, 'numpy')
dux_x = sympy.diff(ux, x)
dux_y = sympy.diff(ux, y)
duy_x = sympy.diff(uy, x)
duy_y = sympy.diff(uy, y)
divu = dux_x + duy_y

sxx = 2 * mu * dux_x + lmbda * divu
sxy = mu * (dux_y + duy_x)
syx = mu * (duy_x + dux_y)
syy = 2 * mu * duy_y + lmbda * divu

sxx_f = sympy.lambdify((x, y), sxx, 'numpy')
sxy_f = sympy.lambdify((x, y), sxy, 'numpy')
syx_f = sympy.lambdify((x, y), syx, 'numpy')
syy_f = sympy.lambdify((x, y), syy, 'numpy')

rhs_x = sympy.diff(sxx, x) + sympy.diff(syx, y)
rhs_y = sympy.diff(sxy, x) + sympy.diff(syy, y)
rhs_x_f = sympy.lambdify((x, y), rhs_x, 'numpy')
rhs_y_f = sympy.lambdify((x, y), rhs_y, 'numpy')


def run_convergence(grid_type):
    u_err = np.zeros((num_refs, len(kappa_list)))
    flux_err = np.copy(u_err)
    
    for iter1, g in enumerate(setup_grids.grid_sequence_fixed_lines(basedim, num_refs, grid_type, pert, subdom_func=chi)):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        for iter2, kappa in enumerate(kappa_list):        
        # Permeability tensor
            char_func_cells = chi(g.cell_centers[0], g.cell_centers[1]) * 1.
            mat_vec = (1 - char_func_cells) + kappa * char_func_cells

            k = fourth_order_tensor.FourthOrderTensor(2, mat_vec, mat_vec)

            # Set type of boundary conditions - Dirichlet
            bound_faces = g.get_boundary_faces()
            bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)

            # MPFA discretization, and system matrix
            stress, bound_stress = mpsa.mpsa(g, k, bound_cond)
            div = fvutils.vector_divergence(g)
            a = div * stress

            # Boundary conditions
            xf = g.face_centers
            char_func_bound = chi(xf[0, bound_faces], xf[1, bound_faces]) * 1
            u_bound = np.zeros((g.dim, g.num_faces))
            u_bound[0, bound_faces] = ux_f(xf[0, bound_faces], xf[1, bound_faces])\
                    / ((1 - char_func_bound) + kappa * char_func_bound)
            u_bound[1, bound_faces] = uy_f(xf[0, bound_faces], xf[1, bound_faces]) \
                    / ((1 - char_func_bound) + kappa * char_func_bound)


            # Right hand side - contribution from the solution and the boundary conditions
            xc = g.cell_centers
            rhs = np.vstack((rhs_x_f(xc[0], xc[1]), 
                             rhs_y_f(xc[0], xc[1]))) * g.cell_volumes
            b = rhs.ravel('F') + div * bound_stress * u_bound.ravel('F')

            # Solve system, derive fluxes
            u_num = scipy.sparse.linalg.spsolve(a, b)
            stress_num = stress * u_num - bound_stress * u_bound.ravel('F')

            ux_num = u_num[::2]
            uy_num = u_num[1::2]
            
            stress_x_num = stress_num[::2]
            stress_y_num = stress_num[1::2]
            # Exact solution
            ux_ex = ux_f(xc[0], xc[1])/ ((1 - char_func_cells) + kappa * char_func_cells)
            uy_ex = uy_f(xc[0], xc[1])/ ((1 - char_func_cells) + kappa * char_func_cells)
            u_ex = np.vstack((ux_ex, uy_ex))
            u_diff = np.vstack((ux_num - ux_ex, uy_num - uy_ex))

            sx_ex_faces = np.vstack((sxx_f(xf[0], xf[1]), sxy_f(xf[0], xf[1])))
            sy_ex_faces = np.vstack((syx_f(xf[0], xf[1]), syy_f(xf[0], xf[1])))

            stress_x_ex = np.sum(g.face_normals * sx_ex_faces, axis=0)
            stress_y_ex = np.sum(g.face_normals * sy_ex_faces, axis=0)
            stress_diff = np.vstack((stress_x_num - stress_x_ex, 
                                     stress_y_num - stress_y_ex))
            stress_ex = np.vstack((stress_x_ex, stress_y_ex))
            
            u_err[iter1, iter2] = np.sqrt(np.sum(g.cell_volumes * u_diff**2)) \
                        / np.sqrt(np.sum(g.cell_volumes * u_ex**2))
            flux_err[iter1, iter2] = np.sqrt(np.sum((g.face_areas ** g.dim) * stress_diff**2))\
                        / np.sqrt(np.sum((g.face_areas ** g.dim) * stress_ex**2))

    return u_err, flux_err

grids = ['cart', 'triangular']

for gr in grids:
    u, f = run_convergence(gr)
    print(u)
    print(f)


[[ 0.38707647  0.51439666  0.52211052]
 [ 0.10514942  0.11284306  0.10794662]
 [ 0.02294892  0.02818847  0.02665215]
 [ 0.00606113  0.00710081  0.00643112]]
[[ 0.22785554  0.22643673  0.23802875]
 [ 0.07617105  0.06993615  0.07181565]
 [ 0.02941291  0.02754268  0.02920611]
 [ 0.01066418  0.01013386  0.01065885]]
[[ 0.19496202  0.37736155  0.37082897]
 [ 0.06394923  0.08809481  0.08384233]
 [ 0.01892896  0.0246903   0.02234049]
 [ 0.00512567  0.00659973  0.00602904]]
[[ 0.20575976  0.19839877  0.2097602 ]
 [ 0.07998824  0.07646041  0.07747881]
 [ 0.0326783   0.03125329  0.0325425 ]
 [ 0.01226182  0.01174649  0.01209849]]
